Lyric Generation using GPT-2 Model<br>
Group: Dawson Sargent, Chenghui Song, Kelvin Yi<br>
Code by: Dawson Sargent<br><br>
Note: Code was ran on Paperspace (as Cheaha was not working) with 45GiB Ram, 8 CPU, 16 GiB GPU

In [1]:
# necessary imports/installs
!pip install transformers --quiet
!pip install datasets --quiet
import torch, os, re, pandas as pd, json
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig
from datasets import Dataset
import tensorflow as tf

In [5]:
# load the lyrics for a particular artist of choice
artist =  "Lana Del Rey"

# code from Chenghui
lyrics = pd.read_csv('lyrics-data.csv') 
lyrics=lyrics.query("language=='en'")
artists = pd.read_csv("artists-data.csv")
lyrics_df = pd.merge(lyrics,artists,left_on="ALink",right_on="Link")
lyrics_df = lyrics_df[["Artist","Genres","Popularity","Songs","SName","Lyric"]]
lyrics_popular=lyrics_df.query(f"Artist=='{artist}'")
lyrics_popular=lyrics_popular.sort_values(['Songs','Artist'], ascending=[False,True])
lyrics_popular.head()
raw_file = f"{artist}_raw_data.csv"
lyrics_popular.to_csv(raw_file)
print("Number of songs from artist:",lyrics_popular.shape[0])


# collect all lyrics from selected artist
lyrics_artist = lyrics_popular[lyrics_popular.Artist==artist]['Lyric'].values.tolist()
text = ""
songs = []
for i in range (len(lyrics_artist)):
    s = lyrics_artist[i]
    # remove unwanted text like [Chorus] or (Chorus)
    s = re.sub("[\(\[].*?[\)\]]", "", s)
    songs.append(s)
    text += lyrics_artist[i]+"\n\n"

with open("lyrics_cleaned.txt",'w', encoding="utf-8") as f:
    for line in songs:
        f.write(line)
        f.write("\n")

        
with open('lyrics_cleaned.txt') as f:
    lines = f.readlines()

section = ''
# remove new line character strings -> [...,'\n',...]
songs = [i for i in songs if i != '\n']
section = ''
sections = []

# collect up lyrics that are ~2000 characters
for text in lines:
    if len(section) < 2000:
        section += text
    else:
        sections.append(section.strip())
        section = ''

df = pd.DataFrame(sections)
ready_file = f"{artist}.csv"
df.to_csv(ready_file,index_label = "id", header= ['text'])

# a look at the data
data = pd.read_csv(raw_file, encoding = 'utf-8',usecols=['Artist','SName','Lyric'])
pd.set_option("display.max_colwidth", None)
data.head(5)

Number of songs from artist: 310


,Artist,SName,Lyric
0,Lana Del Rey,Ride (Monologue),"""I was in the winter of my life\nAnd the men that I met along the road were my only summer.\nAt night, I fell asleep with visions of myself\nDancing and laughing and crying with them.\n\nThree years down the line of being on an endless world tour\nAnd my memories of them were the only things that\nSubstained me and my only real happy times.\n\nI was a singer, not a very popular one,\nWho one had dreams of becoming a beautiful poet -\nBut upon an unfortunate series of events\nSaw those dreams dashed\nAnd divided like a millions stars in the night sky\nThat I wished on over and over again - sparkling and broken.\n\nBut I didn't really mind because I knew\nThat it takes getting everything\nYou ever wanted and then losing it\nTo know what true freedom is.\n\nWhen the people I used to know\nFound out what I had been doing\nHow I had been living - they asked me why.\nBut there's no use in talking to people who have a home,\nThey have no idea what it's like to seek safety I\nIn other people,\nFor home to be wherever you lie your head.\n\nI was always an unusual girl,\nMy mother told me I had a chameleon soul.\nNo morall compass pointing due north, no fixed personality.\nJust an inner indecisiveness\nThat was as wide and as wavering as the ocean.\nAnd if I said that I didn't plan\nFor it to turn out this was, I'd be lying -\nBecause I was born to be the other woman.\n\nI belong to no one - who belonged to everyone,\nWho had nothing, who wanted everything\nWith a fire for every experience\nAnd an obsession for freedom\nThat terrified me to the point I couldn't even talk about -\nAnd pushed me to a nomatic point\nOf madness that both dazzled and dizzied me.""\n\nI've been out on that open road\nYou can be my full time, daddy\nWhite and gold\nSinging blues has been getting old\nYou can be my full time, baby\nHot or cold\n\nDon't break me down\nI've been traveling too long\nI've been trying too hard\nWith one pretty song\n\nI hear the birds on the summer breeze, I drive fast\nI am alone in the night\nBeen trying hard not to get into trouble, but I\nI've got a war in my mind\nSo, I just ride, I just ride\nI just ride, I just ride\nDying young and I'm playing hard\nThat's the way my father made his life an art\nDrink all day and we talk 'til dark\nThat's the way the road doves do it, ride 'til dark\n\nDon't leave me now\nDon't say goodbye\nDon't turn around\nLeave me high and dry\n\nI hear the birds on the summer breeze, I drive fast\nI am alone in the night\nBeen trying hard not to get into trouble, but I\nI've got a war in my mind\nI just ride, I just ride\nI just ride, I just ride\n\nI'm tired of feeling like I'm f*cking crazy\nI'm tired of driving 'til I see stars in my eyes\nI look up to hear myself saying\n""Baby, too much I strive, I just ride""\n\nI hear the birds on the summer breeze, I drive fast\nI am alone in the night\nBeen trying hard not to get into trouble, but I\nI've got a war in my mind\nI just ride, I just ride\nI just ride, I just ride\n\n""Every night I used to pray that I'd find my people -\nAnd finally I did - on the open road.\nWe had nothing to lose, nothing to gain\nNothing we desired anymore -\nExcept to make our lives into a work of art.\n\nLive fast. Die young.\nBe wild, and have fun\nI believe in the country America used to be.\nI believe in the person I want to become.\nI believe in the freedom of the open road.\n\nAnd my motto is the same as ever -\nI believe in the kindness of strangers.\nAnd when I'm at war with myself - I ride. I just ride.\n\nWho are you?\nAre you in touch with all your darkest fantasies?\nHave you created a life for yourself where you're free to experience them?\nI have! I am fucking crazy, but I am free!"""
1,Lana Del Rey,Born To Die,"Feet don't fail me now\nTake me to the finish line\nOh my heart it breaks every step that I take\nBut I'm hoping at the gates\nThey'll tell me that you're mine\n\nWalking through the city streets\nIs i

In [6]:
# reference -> https://www.modeldifferently.com/en/2021/12/generación-de-fake-news-con-gpt-2/

# new defined tokens
bos = '<|BOS|>'
eos = '<|EOS|>'
pad = '<|pad|>'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad}
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# add new defined tokens to the tokenizer
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# configure the model with the new tokens
config = AutoConfig.from_pretrained('gpt2', bos_token_id=tokenizer.bos_token_id, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id, output_hidden_states=False)

# the pre-trained model is loaded with the custom configuration
model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Embedding(50260, 768)

In [7]:
# print out the split up lyrics for training
data = pd.read_csv(ready_file, encoding = 'utf-8', usecols=['text'])

pd.set_option("display.max_colwidth", None)
data.head(3)

,text
0,"""I was in the winter of my life\nAnd the men that I met along the road were my only summer.\nAt night, I fell asleep with visions of myself\nDancing and laughing and crying with them.\n\nThree years down the line of being on an endless world tour\nAnd my memories of them were the only things that\nSubstained me and my only real happy times.\n\nI was a singer, not a very popular one,\nWho one had dreams of becoming a beautiful poet -\nBut upon an unfortunate series of events\nSaw those dreams dashed\nAnd divided like a millions stars in the night sky\nThat I wished on over and over again - sparkling and broken.\n\nBut I didn't really mind because I knew\nThat it takes getting everything\nYou ever wanted and then losing it\nTo know what true freedom is.\n\nWhen the people I used to know\nFound out what I had been doing\nHow I had been living - they asked me why.\nBut there's no use in talking to people who have a home,\nThey have no idea what it's like to seek safety I\nIn other people,\nFor home to be wherever you lie your head.\n\nI was always an unusual girl,\nMy mother told me I had a chameleon soul.\nNo morall compass pointing due north, no fixed personality.\nJust an inner indecisiveness\nThat was as wide and as wavering as the ocean.\nAnd if I said that I didn't plan\nFor it to turn out this was, I'd be lying -\nBecause I was born to be the other woman.\n\nI belong to no one - who belonged to everyone,\nWho had nothing, who wanted everything\nWith a fire for every experience\nAnd an obsession for freedom\nThat terrified me to the point I couldn't even talk about -\nAnd pushed me to a nomatic point\nOf madness that both dazzled and dizzied me.""\n\nI've been out on that open road\nYou can be my full time, daddy\nWhite and gold\nSinging blues has been getting old\nYou can be my full time, baby\nHot or cold\n\nDon't break me down\nI've been traveling too long\nI've been trying too hard\nWith one pretty song\n\nI hear the birds on the summer breeze, I drive fast\nI am alone in the night\nBeen trying hard not to get into trouble, but I"
1,"So, I just ride, I just ride\nI just ride, I just ride\nDying young and I'm playing hard\nThat's the way my father made his life an art\nDrink all day and we talk 'til dark\nThat's the way the road doves do it, ride 'til dark\n\nDon't leave me now\nDon't say goodbye\nDon't turn around\nLeave me high and dry\n\nI hear the birds on the summer breeze, I drive fast\nI am alone in the night\nBeen trying hard not to get into trouble, but I\nI've got a war in my mind\nI just ride, I just ride\nI just ride, I just ride\n\nI'm tired of feeling like I'm f*cking crazy\nI'm tired of driving 'til I see stars in my eyes\nI look up to hear myself saying\n""Baby, too much I strive, I just ride""\n\nI hear the birds on the summer breeze, I drive fast\nI am alone in the night\nBeen trying hard not to get into trouble, but I\nI've got a war in my mind\nI just ride, I just ride\nI just ride, I just ride\n\n""Every night I used to pray that I'd find my people -\nAnd finally I did - on the open road.\nWe had nothing to lose, nothing to gain\nNothing we desired anymore -\nExcept to make our lives into a work of art.\n\nLive fast. Die young.\nBe wild, and have fun\nI believe in the country America used to be.\nI believe in the person I want to become.\nI believe in the freedom of the open road.\n\nAnd my motto is the same as ever -\nI believe in the kindness of strangers.\nAnd when I'm at war with myself - I ride. I just ride.\n\nWho are you?\nAre you in touch with all your darkest fantasies?\nHave you created a life for yourself where you're free to experience them?\nI have! I am fucking crazy, but I am free!""\nFeet don't fail me now\nTake me to the finish line\nOh my heart it breaks every step that I take\nBut I'm hoping at the gates\nThey'll tell me that you're mine\n\nWalking through the city streets\nIs it by mistake or desire?\nI feel so alone on a Friday night\nCan you make it feel like home\nIf I

In [8]:
# add beginning tag and ending tag to each set of lyrics
data['text'] = bos+' '+data['text']+' '+eos

# split up lyrics for training and validation
data_train, data_val = train_test_split(data, train_size = 0.9, random_state = 77)
print(f'There are {len(data_train)} sets of lyrics from {artist} songs for training and {len(data_val)} for validation')


There are 198 sets of lyrics from Lana Del Rey songs for training and 22 for validation


In [10]:
# reference -> https://www.modeldifferently.com/en/2021/12/generación-de-fake-news-con-gpt-2/

# load sets of data
training_dataset = Dataset.from_pandas(data_train[['text']])
validation_dataset = Dataset.from_pandas(data_val[['text']])

def tok_fun(text):
    return tokenizer(text['text'], truncation=True,padding=True)


tokenized_train_dataset = training_dataset.map(
    tok_fun,
    batched=True,
    num_proc=5,
    remove_columns=['text'],
)
tokenized_val_dataset = validation_dataset.map(
    tok_fun,
    batched=True,
    num_proc=5,
    remove_columns=['text'],
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

TRAINING

In [11]:
# reference -> https://www.modeldifferently.com/en/2021/12/generación-de-fake-news-con-gpt-2/

path = './lyric_generator'

# arguments to used for training the model
training_args = TrainingArguments(
    output_dir=path,
    num_train_epochs=3,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=32,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir=path,
    prediction_loss_only=True,
    save_steps=10000)

# defined data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)

# training for model
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset, 
    eval_dataset=tokenized_val_dataset
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 198
  Num Epochs = 3
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 120


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=120, training_loss=17.28941650390625, metrics={'train_runtime': 71.8105, 'train_samples_per_second': 8.272, 'train_steps_per_second': 1.671, 'total_flos': 224991321984000.0, 'train_loss': 17.28941650390625, 'epoch': 3.0})

In [12]:
# sample generated lyrics
# reference -> https://www.modeldifferently.com/en/2021/12/generación-de-fake-news-con-gpt-2/

text ='<|BOS|>'
device = "cuda:0"
input_text = tokenizer.encode(text, return_tensors = 'pt')
input_text = input_text.to(device)
model = model.to(device)
samples = model.generate(
        input_text, 
        max_length= 300,  
        num_return_sequences= 5,
        no_repeat_ngram_size= 2,
        repetition_penalty= 1.5,
        top_p= 0.92,
        temperature= .9,
        do_sample= True,
        top_k= 125)

generated = []
for each in samples:
    text = tokenizer.decode(each, skip_special_tokens=True)
    generated.append(text)
    
print(f"Generated {artist} lyrics")
count = 1
for each in generated:
    print(f"Sequence {count}:")
    count += 1
    print(each)
    print()



Generated Lana Del Rey lyrics
Sequence 1:
You want to die in a woodwind
Even though you do,I'm not dead. You just come through with your loveHappily ever after is it?When I got his gold from the gilded bowl and was looking for another bottle of champagneBarely gone - that's where we wereAll our lives have been lostIn my eyes are yellow because i can't imagine how she would treat meAfter everything so farWhat did he mean...It didn never happen again when nobody knew about what happenedBack there before they made up their mindsThey came back home'The day will be forever spent on this road no matter who wonFor all those hours like these days don´t feel bad let us talk 'til then if anything happensWe think nothing makes sense but fearthe real worldIs more than enoughBut as longlots go bywe've lived hereand nowhere"



Sequence 2:
I can't handle my own painYou think you should be a saint-like being, but it's too lateA summer storm is falling on meBut I must go to the moon and see who wants 

Reference tutorial for code: https://www.modeldifferently.com/en/2021/12/generación-de-fake-news-con-gpt-2/ <br>
Reference for data: https://www.kaggle.com/datasets/neisse/scrapped-lyrics-from-6-genres